In [ ]:
class_name = 'background'

In [ ]:
import nd2
import argparse
import os
import  matplotlib.pyplot as plt
import numpy as np

import pandas as pd
from collections import defaultdict
from datetime import datetime
import shutil

from nis2pyr.reader import read_nd2file

def load_7D(fpath):    
    print(f'Loading from {fpath}')
    with nd2.ND2File(fpath) as nd2file:
        data = read_nd2file(nd2file)
    return data


def backup_fpath(cur_path):
    backup_dir = os.path.dirname(cur_path)
    backup_subdir = 'backup'
    time_token = datetime.now().strftime('%Y-%m-%d_%H:%M:%S')
    backup_fpath = os.path.join(backup_dir, backup_subdir, f'{time_token}_{os.path.basename(cur_path)}')
    os.makedirs(os.path.dirname(backup_fpath), exist_ok=True)
    shutil.copyfile(cur_path, backup_fpath)
    print(f'Backed up {cur_path} to \n {backup_fpath}\n')

def dict_to_str(rawdata_fpath, crop_dict):
    crop_str = ''
    token = rawdata_fpath
    for key in crop_dict:
        for crop in crop_dict[key]:
            crop_str += f'{token},{key},{class_name},{crop[0]},{crop[1]},{crop[2]},{crop[3]}\n'
    return crop_str


In [ ]:
# data_dir = '/group/jug/ashesh/data/Elisa/'
data_dir = '/facility/imganfacusers/Elisa/DIF17/DIF_17_1'
fnames = [x for x in sorted(os.listdir(data_dir)) if x.endswith('0001.nd2')]
len(fnames)

In [ ]:

pd.Series([x.split('_')[-4] for x in fnames]).value_counts()

In [ ]:
# foreground: 5
# background: 0
fpath = os.path.join(data_dir, fnames[0])

In [ ]:

data = load_7D(fpath)
H = data.shape[4]
W = data.shape[5]
content_cropdict = defaultdict(list)
punctor_cropdict = defaultdict(list) 

In [ ]:
print(data.shape)
# np.quantile(data, [0,0.01, 0.995, 1])

In [ ]:
z_idx = 13
ch_idx = 1
plt.imshow(data[0,0,z_idx,ch_idx,...,0], vmax=120)

In [ ]:
# tmp_data = data[0,0,z_idx,ch_idx,...,0]
# mask = tmp_data > 120
# plt.imshow(mask[1000:1500, 1300:2000])

In [ ]:
hs = 0
# he = 1800
he = 800


ws = 1000
# ws =1400
we = 3000
plt.imshow(data[0,0,z_idx,ch_idx,hs:he,ws:we,0], vmax=130)

### Does it look good? 

In [ ]:
content_cropdict

In [ ]:
content_cropdict[z_idx].append([hs, he, ws, we])
content_cropdict

In [ ]:
outputdir = '/group/jug/ashesh/data/Elisa/DIF17'
token = os.path.basename(fpath)
output_fpath = os.path.join(outputdir, f'{class_name}-cropdict.txt')
output_fpath

In [ ]:
def dump_cropdict(output_fpath, crop_dict):
    if not os.path.exists(output_fpath):
        with open(output_fpath, 'w') as f:
            f.write('fname,z_idx,class_name,hs,he,ws,we\n')
        print(f'Created {output_fpath}')
    backup_fpath(output_fpath)
    with open(output_fpath, 'a') as f:
        f.write(dict_to_str(fpath, crop_dict))
    print(f'Written to {output_fpath}')
    return True

if dump_cropdict(output_fpath, content_cropdict):
    content_cropdict = defaultdict(list)


In [ ]:
! cat /group/jug/ashesh/data/Elisa/DIF17/background-cropdict.txt

In [ ]:
df = pd.read_csv('/group/jug/ashesh/data/Elisa/DIF17/background-cropdict.txt')
((df['he'] - df['hs'])* (df['we'] - df['ws'])).sum() / (1000*1000)